<a href="https://colab.research.google.com/github/OneFineStarstuff/OneFineStarstuff/blob/main/Implementing_Differential_Privacy_in_Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install opacus

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from opacus import PrivacyEngine

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define a simple model
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.fc = nn.Linear(784, 10)

    def forward(self, x):
        return self.fc(x)

# Create dummy data
X = torch.randn(50000, 784)
y = torch.randint(0, 10, (50000,))
train_dataset = TensorDataset(X, y)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Initialize model and optimizer
model = MyModel().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Initialize PrivacyEngine
privacy_engine = PrivacyEngine()

# Attach PrivacyEngine to the model, optimizer, and data loader
model, optimizer, train_loader = privacy_engine.make_private(
    module=model,
    optimizer=optimizer,
    data_loader=train_loader,
    noise_multiplier=1.1,
    max_grad_norm=1.0,
)

# Training loop
for epoch in range(10):  # Specify the number of epochs
    for batch in train_loader:
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        optimizer.step()

# Print privacy budget
epsilon = privacy_engine.accountant.get_epsilon(delta=1e-5)
print(f"Epsilon: {epsilon}")